In [ ]:
import os


_logconfpath = './log.conf'

logconfig = """
[loggers]
keys=root

[handlers]
keys=rotatinghandler, timedrotatinghandler

[formatters]
keys=_formatter

[logger_root]
level=DEBUG
handlers=rotatinghandler, timedrotatinghandler

[handler_rotatinghandler]
class=handlers.RotatingFileHandler
level=DEBUG
formatter=_formatter
args=('./app.log', 'a', 1024*1024, 5, 'utf-8')

[handler_timedrotatinghandler]
class=handlers.TimedRotatingFileHandler
level=ERROR
formatter=_formatter
args=('./rotation.log', 'S', 1024*1024, 5, 'utf-8')


[formatter__formatter]
format=[%(process)d-%(thread)d]-%(asctime)s-[%(filename)s:%(lineno)d]-%(levelname)s-%(message)s
"""

f = open(_logconfpath, 'w')
f.write(logconfig)
f.close()


In [ ]:
import logging
import sys

import pandas as pd
import torch
from torch.utils.data import DataLoader

from dataset import MNISTDataset
from model import MNISTResNet
from utils import train, test

logging.config.fileConfig(fname=_logconfpath)
logger = logging.getLogger(__name__)


In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

batch_size = 128
train_loader = DataLoader(MNISTDataset(
    train_df), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(MNISTDataset(
    test_df), batch_size=batch_size, shuffle=False)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MNISTResNet()
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [ ]:
epoch_num = 1
for epoch in range(1, epoch_num + 1):
    train(model, device, train_loader, optimizer)
    error_rate = test(model, device, train_loader)
    logger.info({'epoch': epoch, 'acc': 1 - error_rate})
